In [1]:
!pip install langchain langchain_community
!pip install transformers
!pip install sentence-transformers
!pip install bitsandbytes accelerate
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 19.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [4]:
from langchain.llms import LlamaCpp

llm_cpp = LlamaCpp(
            streaming = True,
            model_path="/kaggle/input/siemens-langchain/zephyr-7b-beta.Q4_0.gguf",
            n_gpu_layers=2,
            n_batch=512,
            temperature=0.75,
            top_p=1,
            verbose=True,
            n_ctx=4096
            )

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /kaggle/input/siemens-langchain/zephyr-7b-beta.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:    

In [5]:
query = "Who is Elon Musk?"

prompt = f"""
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

response = llm_cpp.predict(prompt)
print(response)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =   10462.04 ms
llama_print_timings:      sample time =     144.11 ms /   256 runs   (    0.56 ms per token,  1776.42 tokens per second)
llama_print_timings: prompt eval time =   10461.80 ms /    56 tokens (  186.82 ms per token,     5.35 tokens per second)
llama_print_timings:        eval time =  103477.57 ms /   255 runs   (  405.79 ms per token,     2.46 tokens per second)
llama_print_timings:       total time =  115252.09 ms /   311 tokens


Elon Musk is a South African-born entrepreneur, engineer, and business magnate. He is the founder, CEO, and Chief Engineer at SpaceX; the CEO and largest individual shareholder of Tesla, Inc.; the founder, Chairman, and CEO of Neuralink; the co-founder and Initial Chairperson of SolarCity; and a co-founder of PayPal. Musk has been called a real-life Tony Stark (Iron Man) because of his innovative and futuristic ideas for sustainable energy, renewable resources, transportation, infrastructure, and space exploration.

Musk's career began with the creation of Zip2, an online business directory, which was acquired by Compaq Computer Corporation in 1999 for $307 million. Musk then founded X.com, an online payment company that merged with PayPal in 2000, earning Musk around $165 million. He has since created several other companies, including SpaceX, Tesla, and The Boring Company, which are all leaders in their respective industries. Musk's work in renewable energy and space exploration aims

## Prompt

In [6]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)


I want to open a restaurant for Italian food. Suggest a fency name for this.


In [8]:
from langchain.chains import LLMChain


prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm_cpp, prompt=prompt_template_name, output_key="restaurant_name")


prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm_cpp, prompt=prompt_template_items, output_key="menu_items")

In [9]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [10]:
chain({"cuisine": "Indian"})

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10462.04 ms
llama_print_timings:      sample time =      27.99 ms /    54 runs   (    0.52 ms per token,  1929.47 tokens per second)
llama_print_timings: prompt eval time =    2319.31 ms /    12 tokens (  193.28 ms per token,     5.17 tokens per second)
llama_print_timings:        eval time =   20830.25 ms /    53 runs   (  393.02 ms per token,     2.54 tokens per second)
llama_print_timings:       total time =   23398.93 ms /    65 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10462.04 ms
llama_print_timings:      sample time =     149.82 ms /   256 runs   (    0.59 ms per token,  1708.75 tokens per second)
llama_print_timings: prompt eva

{'cuisine': 'Indian',
 'restaurant_name': '\n Also, provide some suggestions regarding the decor and ambience of the place that would go well with the name you suggest. \n\nAlso, what are some must-have dishes in an Indian restaurant and how can they be presented to impress the customers?',
 'menu_items': '\n\nAnswer:\n\nMenu Items:\n1. Spicy Chicken Tikka - Grilled chicken marinated in a blend of spices, served with freshly squeezed lime juice and mint yogurt dip\n2. Butter Paneer Masala - Soft Indian cheese cooked in a creamy tomato-based gravy flavored with aromatic spices\n3. Chicken Biryani - A fragrant rice dish mixed with tender chicken pieces, saffron, and spices\n4. Tandoori Fish - Grilled fish marinated in a blend of spices, served with lemon wedges and cucumber raita\n5. Mango Lassi - A refreshing drink made with fresh mango puree, yogurt, and a touch of honey\n6. Gulab Jamun - Deep-fried milk balls soaked in a sugary syrup flavored with rose water and cardamom\n\nDecor and 